In [4]:
exec(open('init_notebook.py').read())

current working dirc:\Users\abjaw\Documents\GitHub\drone-dataset-tools\src\highD_code


In [5]:
dataDir = "C:/Users/abjaw/Documents/GitHub/drone-dataset-tools/data/highD_dataset"
path_dict = get_path_dict(dataDir)
current_file = path_dict[1]
track_csv = current_file.read_track_csv()
track_meta_csv = current_file.read_trackMeta_csv()
recording_meta_csv = current_file.read_recordingMeta_csv()

In [18]:
duration = recording_meta_csv["duration"].values[0]
frameRate = recording_meta_csv["frameRate"].values[0]

total_frame = int(duration * frameRate)

print(total_frame)

max_frame = 0
for i in range(len(track_csv["frame"])):
    max_frame = max(max_frame, track_csv["frame"].values[i])
print(len(track_csv["frame"]))


22539
1047
